# [실습] LangChain을 이용한 어플리케이션 개발   

LCEL 구조를 이용하여, 간단한 텍스트를 요약하는 어플리케이션을 만들어 보겠습니다.

In [13]:
!pip install google_genai langchain langchain-community langchain_google_genai pymupdf

  Using cached websockets-15.0.1-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
Using cached websockets-15.0.1-cp312-cp312-win_amd64.whl (176 kB)

   ---------------------------------------- 0/2 [websockets]
   ---------------------------------------- 0/2 [websockets]
   -------------------- ------------------- 1/2 [google_genai]
   -------------------- ------------------- 1/2 [google_genai]
   -------------------- ------------------- 1/2 [google_genai]
   ---------------------------------------- 2/2 [google_genai]



## Gemini API 준비하기


Google API 키를 등록하고 입력합니다.   
구글 계정 로그인 후 https://aistudio.google.com  에 접속하면, API 키 생성이 가능합니다.

In [2]:
import os
os.environ['GOOGLE_API_KEY'] = 'AIxxx'

from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# rate limiter를 LLM에 적용
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    rate_limiter=rate_limiter,
    temperature = 0.5,
    max_tokens = 2048
)

PDF 파일 준비하기   
임의의 PDF 파일을 다운로드하여 준비합니다.   
(예시 PDF 파일을 사용하실 분은 첨부된 PDF 파일을 활용해 주세요!)

In [4]:
path_material = 'example.pdf'
# 자유롭게 경로 변경해서 실행하셔도 됩니다!

랭체인에서는 데이터를 `Document` 클래스로 처리합니다.   
데이터의 형식에 따라 적절한 document_loader를 불러와서 사용할 수 있습니다.   

이번 실습에서는 PDF를 불러오는 가장 간단한 로더인 PyMuPDFLoader를 사용합니다.

In [8]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(path_material)
# 페이지별로 저장
pages = loader.load()
print("# Number of Pages:", len(pages))

# Number of Pages: 32


파일이 너무 긴 경우,일부만 선택하여 요약할 수 있습니다.

In [9]:
pages = pages[:11]

In [10]:
pages[10]


Document(metadata={'producer': 'pikepdf 8.15.1', 'creator': 'arXiv GenPDF (tex2pdf:)', 'creationdate': '', 'source': 'example.pdf', 'file_path': 'example.pdf', 'total_pages': 32, 'format': 'PDF 1.5', 'title': 'MCPEval: Automatic MCP-based Deep Evaluation for AI Agent Models', 'author': 'Zhiwei Liu; Jielin Qiu; Shiyu Wang; Jianguo Zhang; Zuxin Liu; Roshan Ram; Haolin Chen; Weiran Yao; Huan Wang; Shelby Heinecke; Silvio Savarese; Caiming Xiong', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 10}, page_content='A\nComprehensive Experiments and Results\nWe present a comprehensive evaluation of 10 state-of-the-art LLM models across 5 diverse domains using\nthe MCPEval framework. Our analysis encompasses 5k trajectory records and 5k completion records\nwith detailed individual task analysis, plus 50 model-domain combinations from systematic LLM judger\nevaluation, representing the most extensive evaluation of LLM tool-use capabilities 

PDF 파일은 페이지별 Document를 저장합니다.   
요약을 수행하기 위해, 전체 텍스트를 하나의 Document에 합칩니다.

In [11]:
from langchain_core.documents import Document
# Document 클래스 만들기

corpus = Document(page_content='')
for page in pages:
    corpus.page_content += page.page_content + '\n'

corpus.page_content = corpus.page_content.replace('\n\n','\n')
len(corpus.page_content)

43650

LLM에 처리하기 전, 토큰 수를 체크합니다.

In [14]:
from google import genai
from google.genai import types

client = genai.Client()


response = client.models.count_tokens(
    model='gemini-2.0-flash',
    contents=corpus.page_content
)

print("Prompt tokens:",response.total_tokens)

Prompt tokens: 14155


요약 체인을 만들고 구성합니다.

## [실습] 요약 체인 만들기

`corpus.page_content`를 입력으로 받는 요약 체인을 만들고 실행하세요.

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
summary_prompt = ChatPromptTemplate(
    [('user', '''
당신은 LLM/AI 전문가입니다.
LLM에 대한 최신 모델의 논문이 주어집니다.
전문 개발자를 대상으로 이를 설명한다고 가정하고,
모델의 구조에 집중해서 논문의 내용을 요약하세요.

전체는 5문단으로 하고, 문단별 4~8개 문장으로 구성하세요.
---
{text}''')

    ]

)
summary_chain = summary_prompt | llm | StrOutputParser()

summary = summary_chain.invoke(corpus.page_content)
summary

'## MCPEval: AI 에이전트 모델을 위한 자동 MCP 기반 심층 평가\n\n본 논문에서는 LLM 기반 AI 에이전트의 자동 심층 평가를 위한 MCPEval 프레임워크를 소개합니다. 기존의 평가 방법은 정적인 벤치마크와 수동 데이터 수집에 의존하여 실제적인 평가에 제한이 있었습니다. MCPEval은 MCP(Model Context Protocol)를 기반으로 하여 다양한 도메인에서 LLM 에이전트의 엔드 투 엔드 작업 생성 및 심층 평가를 자동화합니다. 이를 통해 평가 파이프라인 구축에 필요한 수동 노력을 줄이고, 표준화된 메트릭을 제공하며, 에이전트의 도구와 원활하게 통합됩니다.\n\nMCPEval 프레임워크는 크게 세 가지 주요 단계로 구성됩니다. 첫째, 작업 생성 단계에서는 MCP 서버의 도구 호출 메서드를 활용하여 도구 사양을 수집하고, Task-LLM을 통해 상세한 작업 지침을 생성합니다. 둘째, 작업 검증 단계에서는 프론티어 에이전트가 MCP 클라이언트로서 MCP 서버와 상호 작용하며 생성된 작업을 실행합니다. 성공적인 실행 궤적은 검증된 작업과 해당 정답 궤적으로 이어지며, 실패 시에는 작업 업데이트 요청을 통해 작업 설명을 개선합니다. 이러한 반복적인 검증 및 개선 프로세스를 통해 고품질의 작업 데이터셋을 구축합니다.\n\n모델 평가 단계에서는 평가 대상 모델을 MCP 클라이언트로 설정하고 검증된 작업 세트를 완료하도록 합니다. 수집된 궤적은 두 가지 관점에서 분석됩니다. 첫째, 도구 호출 매칭은 모델의 도구 사용을 정답 궤적과 엄격하게 비교합니다. 둘째, LLM 심판은 계획, 실행 흐름, 컨텍스트 인식 등 다양한 차원을 평가합니다. MCPEval은 이러한 분석 결과를 결합하여 각 에이전트 모델의 강점, 약점 및 성능을 상세히 설명하는 포괄적인 보고서를 자동 생성합니다.\n\nMCPEval은 다양한 아키텍처와 기능을 가진 10개의 모델을 대상으로 5개의 실제 도메인에서 실험을 진행했습니다. 평가에는 GPT-4o, GPT-4o-mini, M

Gemini 2.0 Flash의 Context Window는 1M이므로 전체를 모두 하나의 컨텍스트로 입력해도 되지만,   
Context가 짧은 모델들의 경우, 전체를 분할하여 요약 작업을 수행할 수 있습니다.

**Map-Reduce** 방식의 요약을 만들어 보겠습니다.   

Map-Reduce는 텍스트를 청크로 분할하고, 청크별 요약을 생성한 뒤  
전체 요약문을 합쳐 프롬프트로 넣고 최종 요약문을 생성하는 방식입니다.

문서를 청크로 나누기 위해, RecursiveCharacterTextSplitter를 사용합니다.

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20000,
    chunk_overlap=4000,
)

chunks = text_splitter.split_documents([corpus])
print(len(chunks))

3


Map: 청크별 요약을 생성합니다.

In [17]:
# Map 과정 : 각 문서에 대해 요약을 생성합니다.
from tqdm import tqdm

map_prompt = ChatPromptTemplate([
    ('system', '''논문의 일부가 주어집니다.
해당 내용을 읽고 한국어로 요약하세요.
요약은 5개의 문단과 문단별 4~8개의 문장으로 작성하세요.
'''),
    ('user', '''{text}''')])

raw_summaries = []

map_chain  = map_prompt | llm | StrOutputParser()


for i in tqdm(range(len(chunks))):
    response = map_chain.invoke(chunks[i].page_content)

    raw_summaries.append(response)

    print('\n#',i)
    print(response)
    print('===========================')

 33%|███▎      | 1/3 [00:06<00:13,  6.90s/it]


# 0
## MCPEval 논문 요약

**서론:**
최근 대규모 언어 모델(LLM) 기반 지능형 에이전트의 급증으로 인해, 확장 가능하고 강력한 평가 프레임워크의 필요성이 대두되고 있습니다. 기존 방법들은 정적인 벤치마크와 노동 집약적인 데이터 수집에 의존하여 실제적인 평가에 한계가 있습니다. 이에 본 논문에서는 다양한 도메인에서 LLM 에이전트의 엔드 투 엔드 작업 생성 및 심층 평가를 자동화하는 오픈 소스 모델 컨텍스트 프로토콜(MCP) 기반 프레임워크인 MCPEval을 소개합니다. MCPEval은 표준화된 지표를 제공하고, 에이전트의 기본 도구와 원활하게 통합되며, 평가 파이프라인 구축에 필요한 수동 노력을 제거합니다.

**관련 연구:**
LLM 평가 방식은 정적 벤치마크에서 벗어나 에이전트 기반의 동적인 방식으로 진화하고 있습니다. 초기에는 HELM, BIG-bench, MMLU와 같은 정적 벤치마크가 사용되었지만, 실제 환경과의 상호작용을 제대로 반영하지 못한다는 한계가 있었습니다. 이후 MT-Bench, AgentBoard, AgentBench와 같은 대화형 및 에이전트 기반 벤치마크가 등장했지만, 여전히 도구 통합이 부족했습니다. 이러한 문제를 해결하기 위해 MCP가 LLM과 외부 시스템 간의 상호 작용을 관리하는 핵심 표준으로 떠올랐으며, MCP-Radar, MCPWorld와 같은 프레임워크가 개발되었습니다.

**MCPEval 프레임워크:**
MCPEval은 작업 생성, 검증 및 모델 평가로 구성된 평가 워크플로우를 채택하여 LLM 에이전트의 효율적이고 확장 가능한 평가를 지원합니다. 작업 생성은 MCP 서버에서 도구 호출 메서드를 호출하여 도구 사양을 프롬프트로 수집하는 것으로 시작합니다. Task-LLM은 도구 호출에 필요한 정보가 포함되도록 상세한 작업 지침을 생성합니다. 생성된 작업이 도구 매개변수를 채우는 데 필요한 모든 정보를 포함하지 않을 수 있으므로, 작업 검증을 통해 고품질 작업을 생성하고 실제 실행 궤적을 수집합니다.



 67%|██████▋   | 2/3 [00:13<00:06,  6.89s/it]


# 1
다음은 제공된 논문 발췌문의 5개 문단 요약입니다.

**1. LLM 평가 결과 개요**
본 논문은 LLM(Large Language Model)의 MCP(Model Context Protocol) 환경에서의 성능을 평가한 결과를 제시합니다.  평가에는 다양한 모델이 사용되었으며, 각 모델은 trajectory(Traj)와 completion(Comp) 두 가지 측면에서 평가되었습니다.  평가 결과는 금융, Airbnb, 의료, 스포츠, 국립공원 등 다양한 도메인에 걸쳐 제시되었으며, 각 도메인별 평균 점수도 함께 제공됩니다.  OpenAI의 O3 모델이 계획, 요구사항 커버리지, 완전성 측면에서 뛰어난 성능을 보였으며, gpt4o 모델들은 효과적인 계획과 도구 호출 접근 방식을 결합하여 좋은 결과를 나타냈습니다.  반면, mistral-small-24b 및 qwen3 모델과 같은 오픈 소스 모델은 성능 격차와 모델별 도구 사용 패턴의 차이로 인해 낮은 점수를 받았습니다.

**2. 도메인별 성능 분석**
도메인별 성능 분석 결과, 의료 분야가 가장 높은 성능을 보였고, Airbnb, 스포츠, 금융, 국립공원 순으로 나타났습니다.  의료 분야는 표준화된 데이터와 API 출력과 사용자 요구 간의 강력한 일치 덕분에 우수한 성적을 거두었습니다.  반면, 국립공원 분야는 복잡한 지리적 데이터 통합으로 인해 어려움을 겪었습니다.  금융 분야는 trajectory 점수는 중간 수준이지만 completion 점수가 높은 반면, Airbnb는 큰 격차를 보여 종합적인 결과 생성에 어려움이 있음을 시사합니다.  의료 및 Airbnb 분야는 작업 분포가 가장 높아 분석의 신뢰성을 높였습니다.  전반적으로, 도메인별 격차는 API 설계 품질과 작업 요구 사항의 차이를 반영합니다.

**3. 성능 격차 분석**
모델 전반적으로 trajectory 실행이 completion보다 더 나은 성능을 보이는 일관된 추세가 나타났습니다.  이는 LLM의 종합 능력에 아직 개선의 여

100%|██████████| 3/3 [00:19<00:00,  6.58s/it]


# 2
다음은 주어진 논문 일부를 5개의 문단으로 요약한 내용입니다. 각 문단은 4~8개의 문장으로 구성되어 있습니다.

이 논문은 대규모 언어 모델(LLM)이 실제 소프트웨어 엔지니어링 문제를 해결할 수 있는지를 평가하기 위한 연구입니다. 다양한 LLM 모델들을 대상으로 여러 도메인에서 툴 사용 능력을 평가하는 MCPEval 프레임워크를 사용하여 광범위한 실험을 수행했습니다. 평가는 5,000개의 궤적 기록과 5,000개의 완료 기록을 포함하며, LLM 심판 평가를 통해 50개의 모델-도메인 조합을 체계적으로 분석하여 LLM의 툴 사용 능력에 대한 포괄적인 평가를 제공합니다.

실험 설정에는 OpenAI 모델 7개(GPT-4o, GPT-4o-mini, GPT-4.1-mini, GPT-4.1-nano, O3, O3-mini, O4-mini)와 오픈 소스 모델 3개(Mistral-Small-24B, Qwen3-32B, Qwen3-30B-A3B)가 포함되었습니다. 평가는 금융, 의료, Airbnb, 스포츠, 국립 공원이라는 5가지 다양한 응용 분야에서 수행되었습니다. 각 도메인에는 주가, 의료 용어 검색, 숙소 검색, 팀 통계, 공원 정보와 같은 특정 도구들이 사용되었습니다. 이러한 다양한 모델과 도메인을 통해 LLM의 일반적인 툴 사용 능력을 평가할 수 있습니다.

AI 에이전트의 능력을 평가하기 위해 툴 호출 성능과 LLM 심판 성능이라는 두 가지 평가 기준을 사용했습니다. 툴 호출 성능은 실제 정답과 비교하여 예측된 툴 사용의 정확성을 측정합니다. LLM 심판 성능은 루브릭 기반 판단을 사용하여 실행 궤적과 작업 완료의 전반적인 품질을 평가합니다. 이 두 가지 기준은 정확한 운영 평가와 높은 수준의 행동 평가를 가능하게 합니다.

툴 호출 평가는 MCP 모델 평가기의 분석 명령을 사용하여 수행됩니다. 이 시스템은 엄격한 매칭 프로토콜과 유연한 매칭 프로토콜을 모두 사용하여 에이전트의 예측된 툴 사용과 실제 정답을 비교합니다. 엄격한 매칭은 툴 이름, 매개변

In [18]:
gathered_summaries = '\n---\n'.join(raw_summaries)

print(gathered_summaries)

## MCPEval 논문 요약

**서론:**
최근 대규모 언어 모델(LLM) 기반 지능형 에이전트의 급증으로 인해, 확장 가능하고 강력한 평가 프레임워크의 필요성이 대두되고 있습니다. 기존 방법들은 정적인 벤치마크와 노동 집약적인 데이터 수집에 의존하여 실제적인 평가에 한계가 있습니다. 이에 본 논문에서는 다양한 도메인에서 LLM 에이전트의 엔드 투 엔드 작업 생성 및 심층 평가를 자동화하는 오픈 소스 모델 컨텍스트 프로토콜(MCP) 기반 프레임워크인 MCPEval을 소개합니다. MCPEval은 표준화된 지표를 제공하고, 에이전트의 기본 도구와 원활하게 통합되며, 평가 파이프라인 구축에 필요한 수동 노력을 제거합니다.

**관련 연구:**
LLM 평가 방식은 정적 벤치마크에서 벗어나 에이전트 기반의 동적인 방식으로 진화하고 있습니다. 초기에는 HELM, BIG-bench, MMLU와 같은 정적 벤치마크가 사용되었지만, 실제 환경과의 상호작용을 제대로 반영하지 못한다는 한계가 있었습니다. 이후 MT-Bench, AgentBoard, AgentBench와 같은 대화형 및 에이전트 기반 벤치마크가 등장했지만, 여전히 도구 통합이 부족했습니다. 이러한 문제를 해결하기 위해 MCP가 LLM과 외부 시스템 간의 상호 작용을 관리하는 핵심 표준으로 떠올랐으며, MCP-Radar, MCPWorld와 같은 프레임워크가 개발되었습니다.

**MCPEval 프레임워크:**
MCPEval은 작업 생성, 검증 및 모델 평가로 구성된 평가 워크플로우를 채택하여 LLM 에이전트의 효율적이고 확장 가능한 평가를 지원합니다. 작업 생성은 MCP 서버에서 도구 호출 메서드를 호출하여 도구 사양을 프롬프트로 수집하는 것으로 시작합니다. Task-LLM은 도구 호출에 필요한 정보가 포함되도록 상세한 작업 지침을 생성합니다. 생성된 작업이 도구 매개변수를 채우는 데 필요한 모든 정보를 포함하지 않을 수 있으므로, 작업 검증을 통해 고품질 작업을 생성하고 실제 실행 궤적을 수집합니다.

**실험 

In [19]:
# Reduce 과정 : 각 문서의 요약을 하나로 합칩니다.
reduce_prompt = ChatPromptTemplate([
    ('system', '''당신은 인공지능과 거대 언어 모델의 전문가입니다.
LLM 논문에 대한 요약문의 리스트가 주어집니다.
이를 읽고, 전체 주제를 포함하는 최종 요약을 작성하세요.
요약은 5개의 문단과 문단별 4-8개의 문장으로 작성하세요.
'''),
    ('user', '''{text}
---
Summary:
''')])

reduce_chain = reduce_prompt | llm | StrOutputParser()

summary = reduce_chain.invoke(gathered_summaries)

print(summary)

이 논문은 LLM 에이전트의 실제 소프트웨어 엔지니어링 문제 해결 능력을 평가하기 위해 MCPEval 프레임워크를 사용하여 다양한 모델을 대상으로 광범위한 실험을 수행했습니다. 5,000개의 궤적 기록과 완료 기록을 분석하고, LLM 심판 평가를 통해 50개의 모델-도메인 조합을 체계적으로 평가하여 LLM의 툴 사용 능력에 대한 포괄적인 평가를 제공합니다. 이 연구는 LLM이 실제 환경에서 복잡한 작업을 수행할 수 있는 잠재력을 탐구하고, 그 한계점을 명확히 밝히는 데 중요한 역할을 합니다.

실험에는 OpenAI 모델 7개와 오픈 소스 모델 3개가 포함되었으며, 금융, 의료, Airbnb, 스포츠, 국립 공원이라는 5가지 다양한 응용 분야에서 평가가 진행되었습니다. 각 도메인에는 주가, 의료 용어 검색, 숙소 검색, 팀 통계, 공원 정보와 같은 특정 도구들이 사용되어 LLM의 툴 사용 능력을 다각적으로 평가했습니다. 다양한 모델과 도메인을 통해 LLM이 실제 문제를 해결하는 데 필요한 다양한 기술과 지식을 평가하고, 특정 분야에서의 강점과 약점을 파악할 수 있었습니다.

AI 에이전트의 능력을 평가하기 위해 툴 호출 성능과 LLM 심판 성능이라는 두 가지 평가 기준을 사용했습니다. 툴 호출 성능은 실제 정답과 비교하여 예측된 툴 사용의 정확성을 측정하고, LLM 심판 성능은 루브릭 기반 판단을 사용하여 실행 궤적과 작업 완료의 전반적인 품질을 평가했습니다. 이러한 다각적인 평가 접근 방식은 LLM의 툴 사용 능력에 대한 정확하고 포괄적인 평가를 가능하게 합니다.

툴 호출 평가는 MCP 모델 평가기의 분석 명령을 사용하여 수행되며, 엄격한 매칭 프로토콜과 유연한 매칭 프로토콜을 모두 사용하여 에이전트의 예측된 툴 사용과 실제 정답을 비교합니다. 엄격한 매칭은 툴 이름, 매개변수 값, 실행 순서에 대한 정확한 일치를 요구하며, 유연한 매칭은 매개변수 값에 대한 유사성 임계값과 툴 순서에 대한 유사성 임계값을 적용하여 부분적인 점수를 허용합니다. 이러한 다양한 